# <center> Test Allociné

## Infos

Pour la recherche de similarité, une similarité cosinus pour les vecteurs embedding, mais dans le cas de très gros volumes on peut faire un Knn approximatif.

Voir aussi FAISS, Annoy, HNSW, ScaNN

ANN: approximate nearest neighbour

Explorer doc2vec


## Imports

In [115]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import spacy
from datasets import load_dataset

import faiss
#pip install faiss-cpu

## Get Data

In [3]:
nlp = spacy.load("fr_core_news_sm")

train_dataset = load_dataset("tblard/allocine", split="train")
df = pd.DataFrame(train_dataset['review'][:100], columns = ['review'])
df

,review
0,Si vous cherchez du cinéma abrutissant à tous ...
1,"Trash, re-trash et re-re-trash...! Une horreur..."
2,"Et si, dans les 5 premières minutes du film, l..."
3,Mon dieu ! Quelle métaphore filée ! Je suis ab...
4,"Premier film de la saga Kozure Okami, ""Le Sabr..."
...,...
95,"Sans doute le pire film que j'ai vu, le ridicu..."
96,"typique des années 90, un peu daté mais pas ma..."
97,Une réalisation pauvre du coup je mettais tous...
98,Ce navet est affligeant tellement il est mauva...


## Tokenisation - lemmatisation

In [4]:
print("Nb reviews:", len(train_dataset['review']))
print(train_dataset[0]['review'])

Nb reviews: 160000
Si vous cherchez du cinéma abrutissant à tous les étages,n'ayant aucune peur du cliché en castagnettes et moralement douteux,"From Paris with love" est fait pour vous.Toutes les productions Besson,via sa filière EuropaCorp ont de quoi faire naître la moquerie.Paris y est encore une fois montrée comme une capitale exotique,mais attention si l'on se dirige vers la banlieue,on y trouve tout plein d'intégristes musulmans prêts à faire sauter le caisson d'une ambassadrice américaine.Nauséeux.Alors on se dit qu'on va au moins pouvoir apprécier la déconnade d'un classique buddy-movie avec le jeune agent aux dents longues obligé de faire équipe avec un vieux lou complètement timbré.Mais d'un côté,on a un Jonathan Rhys-meyers fayot au possible,et de l'autre un John Travolta en total délire narcissico-badass,crâne rasé et bouc proéminent à l'appui.Sinon,il n'y a aucun scénario.Seulement,des poursuites débiles sur l'autoroute,Travolta qui étale 10 mecs à l'arme blanche en 8 mou

In [7]:
def get_lemmas(doc):
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

def review_with_lemmas(doc):
    return ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

df['token'] = df['review'].apply(nlp)
df['lemma'] = df['token'].apply(get_lemmas)
df['review_lemma'] = df['token'].apply(review_with_lemmas)
df

,review,token,lemma,review_lemma
0,Si vous cherchez du cinéma abrutissant à tous ...,"(Si, vous, cherchez, du, cinéma, abrutissant, ...","[chercher, cinéma, abrutissant, étage, aucun, ...",chercher cinéma abrutissant étage aucun peur c...
1,"Trash, re-trash et re-re-trash...! Une horreur...","(Trash, ,, re-trash, et, re-re-trash, ..., !, ...","[trash, re-trash, re-re-trash, horreur, nom, i...",trash re-trash re-re-trash horreur nom imagine...
2,"Et si, dans les 5 premières minutes du film, l...","(Et, si, ,, dans, les, 5, premières, minutes, ...","[5, premier, minute, film, pathétique, maquett...",5 premier minute film pathétique maquette trai...
3,Mon dieu ! Quelle métaphore filée ! Je suis ab...,"(Mon, dieu, !, Quelle, métaphore, filée, !, Je...","[dieu, métaphore, filée, abasourdi, réellement...",dieu métaphore filée abasourdi réellement ici ...
4,"Premier film de la saga Kozure Okami, ""Le Sabr...","(Premier, film, de, la, saga, Kozure, Okami, ,...","[film, saga, Kozure, Okami, sabre, vengeance, ...",film saga Kozure Okami sabre vengeance bon fil...
...,...,...,...,...
95,"Sans doute le pire film que j'ai vu, le ridicu...","(Sans, doute, le, pire, film, que, j', ai, vu,...","[doute, pire, film, ridicule, atteindre, somme...",doute pire film ridicule atteindre sommet ici ...
96,"typique des années 90, un peu daté mais pas ma...","(typique, des, années, 90, ,, un, peu, daté, m...","[typique, année, 90, dater, mal, suspense, bie...",typique année 90 dater mal suspense bien jouer...
97,Une réalisation pauvre du coup je mettais tous...,"(Une, réalisation, pauvre, du, coup, je, metta...","[réalisation, pauvre, coup, mettre, espoir, in...",réalisation pauvre coup mettre espoir interpré...
98,Ce navet est affligeant tellement il est mauva...,"(Ce, navet, est, affligeant, tellement, il, es...","[navet, affliger, mauver, scène, filmer, corre...",navet affliger mauver scène filmer correctemen...


In [8]:
for item in df['review_lemma'].values:
    print(item)

chercher cinéma abrutissant étage aucun peur cliché castagnette moralement douteux,"From Paris with lover production Besson filière europacorp faire naître moquerie Paris fois montrer capitale exotique attention diriger banlieue trouver plein intégriste musulman prêt faire sauter caisson ambassadeur américain Nauséeux pouvoir apprécier déconnade classique buddy movie jeune agent dent long obligé faire équipe vieux lou complètement timbrer côté jonathan Rhys meyer fayot John Travolta total délir narcissico badas crâne raser bouc proéminent appui aucun scénario poursuite débile autoroute travolta étale 10 mec arme blanc 8 mouvements(!!)ou laisse associer faire démolir tronche scruter jumelle ca plaisir coupable hénaurme",c' daube droite ligner transporteur","taken"ou banlieue 13
trash re-trash re-re-trash horreur nom imaginer vous 20 premier minute orange Mécanique dilaté 70 minute bande vhs pourrave revisiter korine sauc année 2000 dandy punk Kubrick laisser place papy lubrique déguiser

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(train_dataset[:6]['review'])
vectors.todense()

## **Similarity Search**

In [ ]:
similarities = cosine_similarity(vectors, vectors)
# top_indices = similarities.argsort()[0][-5:]
similarities

array([2, 1, 0])

In [ ]:
test_similarity = train_dataset[:6]['review'] + ["Si vous cherchez du cinéma abrutissant"]
print(test_similarity)

vectors = vectorizer.fit_transform(test_similarity)
similarities = cosine_similarity(vectors, vectors)
similarities

In [65]:
test_similarity = ["Un avion est en train de décoller.",
          "Un homme joue d'une grande flûte.",
          "Un homme étale du fromage râpé sur une pizza.",
          "Une personne jette un chat au plafond.",
          "Une personne est en train de plier un morceau de papier.",
          ]

vectors = vectorizer.fit_transform(test_similarity)
similarities = cosine_similarity(vectors, vectors)
similarities

array([[1.        , 0.05043642, 0.03851261, 0.04532326, 0.53546271],
       [0.05043642, 1.        , 0.21756628, 0.11661921, 0.0898525 ],
       [0.03851261, 0.21756628, 1.        , 0.08904894, 0.06861022],
       [0.04532326, 0.11661921, 0.08904894, 1.        , 0.17727473],
       [0.53546271, 0.0898525 , 0.06861022, 0.17727473, 1.        ]])

## Embedding

In [38]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoModel, AutoTokenizer

In [ ]:
model = SentenceTransformer(r"C:\Users\Utilisateur\Documents\mytheque\Models\sentence-camembert-large", local_files_only=True)

In [15]:
model.encode(train_dataset[0]['review'])

array([ 0.19306128, -0.11996752, -0.3786842 , ...,  0.12942207,
        0.2684721 ,  0.10108169], shape=(1024,), dtype=float32)

In [ ]:
df['embedding'] = df['review'].apply(lambda w : model.encode(w))

In [19]:
df[['review', 'embedding']]

,review,embedding
0,Si vous cherchez du cinéma abrutissant à tous ...,"[0.19306128, -0.11996752, -0.3786842, 0.047587..."
1,"Trash, re-trash et re-re-trash...! Une horreur...","[-0.01557752, -0.18321595, -0.08755206, 0.1351..."
2,"Et si, dans les 5 premières minutes du film, l...","[-0.2778221, -0.15710041, -0.3382795, 0.167760..."
3,Mon dieu ! Quelle métaphore filée ! Je suis ab...,"[0.23153108, -0.10687427, -0.1058496, 0.408893..."
4,"Premier film de la saga Kozure Okami, ""Le Sabr...","[-0.16487095, 0.38755867, -0.066273704, 0.1954..."
...,...,...
95,"Sans doute le pire film que j'ai vu, le ridicu...","[-0.14285047, -0.06474736, -0.22349982, 0.5060..."
96,"typique des années 90, un peu daté mais pas ma...","[0.17517376, -0.054622572, -0.22175737, 0.1697..."
97,Une réalisation pauvre du coup je mettais tous...,"[0.1267551, -0.4698259, -0.14230217, -0.020845..."
98,Ce navet est affligeant tellement il est mauva...,"[0.0035572576, -0.27487898, -0.23988113, -0.22..."


In [76]:
sentences = ["Un avion est en train de décoller.",
          "Un homme joue d'une grande flûte.",
          "Un homme étale du fromage râpé sur une pizza.",
          "Une personne jette un chat au plafond.",
          "Une personne est en train de plier un morceau de papier.",
          "Un italien prépare le repas avec du fromage"
          ]

embeddings = model.encode(sentences)
print(embeddings.shape)
embeddings

(6, 1024)


array([[ 0.11892722,  0.46865812, -0.62379724, ...,  0.46460527,
         0.2039351 ,  0.07192303],
       [-0.37721995,  0.24875322,  0.1830102 , ...,  0.35383818,
         0.11625216,  0.09876025],
       [-0.16000931, -0.01209719,  0.03265126, ..., -0.05747368,
        -0.41359037,  0.15602526],
       [ 0.23739465, -0.5312759 ,  0.1390972 , ..., -0.46047148,
         0.14653896, -0.5297922 ],
       [ 0.32796627,  0.0279403 , -0.3251529 , ...,  0.6037205 ,
        -0.01314403,  0.16631809],
       [-0.66081023, -0.16198173, -0.22883216, ..., -0.11857213,
        -0.43039393,  0.12713504]], shape=(6, 1024), dtype=float32)

## **Similarity Search**

In [ ]:
similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)

for idx_i, sentence1 in enumerate(sentences):
    print(sentence1)
    for idx_j, sentence2 in enumerate(sentences):
        print(f" - {sentence2: <30}: {similarities[idx_i][idx_j]:.4f}")

In [ ]:
# Similarity from Sklearn
cosine_similarity(vectors, vectors)

array([[1.        , 0.05043642, 0.03851261, 0.04532326, 0.53546271],
       [0.05043642, 1.        , 0.21756628, 0.11661921, 0.0898525 ],
       [0.03851261, 0.21756628, 1.        , 0.08904894, 0.06861022],
       [0.04532326, 0.11661921, 0.08904894, 1.        , 0.17727473],
       [0.53546271, 0.0898525 , 0.06861022, 0.17727473, 1.        ]])

## Comparison of similarity search

We compare similarity search from TF-IDF and from embedding

In [129]:
sentences = ["Un avion est en train de décoller.",
          "Un homme joue d'une grande flûte.",
          "Un homme étale du fromage râpé sur une pizza.",
          "Une personne jette un chat au plafond.",
          "Une personne est en train de plier un morceau de papier.",
          "Un italien prépare le repas avec du fromage"
          ]

In [130]:
vectors = vectorizer.fit_transform(sentences)
similarities = cosine_similarity(vectors, vectors)
similarities

array([[1.        , 0.04338402, 0.03478563, 0.039009  , 0.53882999,
        0.03485039],
       [0.04338402, 1.        , 0.2319948 , 0.11693662, 0.08942492,
        0.03749623],
       [0.03478563, 0.2319948 , 1.        , 0.09376067, 0.07170157,
        0.23525734],
       [0.039009  , 0.11693662, 0.09376067, 1.        , 0.17889004,
        0.03371496],
       [0.53882999, 0.08942492, 0.07170157, 0.17889004, 1.        ,
        0.02578284],
       [0.03485039, 0.03749623, 0.23525734, 0.03371496, 0.02578284,
        1.        ]])

In [103]:
embeddings = model.encode(sentences)
similarities = model.similarity(embeddings, embeddings)
similarities

tensor([[ 1.0000, -0.0397, -0.0080,  0.0912,  0.0802, -0.0068],
        [-0.0397,  1.0000,  0.0366, -0.0030, -0.0276,  0.0167],
        [-0.0080,  0.0366,  1.0000,  0.0208,  0.1100,  0.6652],
        [ 0.0912, -0.0030,  0.0208,  1.0000,  0.1168, -0.0163],
        [ 0.0802, -0.0276,  0.1100,  0.1168,  1.0000,  0.0678],
        [-0.0068,  0.0167,  0.6652, -0.0163,  0.0678,  1.0000]])

## FAISS

**With TF-IDF**

In [ ]:
print(type(vectors))
dense_matrix = vectors.toarray().astype('float32')
print(type(dense_matrix))

dim = dense_matrix.shape[1]
print("dim", dim)

index = faiss.IndexFlatIP(dim)  # Produit Scalaire
index.add(dense_matrix)

# Recherche
query_embedding = model.encode(["chien"]).astype(np.float32)
distances, indices = index.search(query_embedding, k=3)
# distances, indices

In [ ]:
# Recherche
query_embedding = model.encode(["chien"]).astype(np.float32)
distances, indices = index.search(query_embedding, k=6)  # Top 10 résultats
distances, indices

In [ ]:
# Création de l'index FAISS
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)  # Produit Scalaire
index.add(embeddings.astype(np.float32))

# Recherche
query_embedding = model.encode(["chien"]).astype(np.float32)
distances, indices = index.search(query_embedding, k=6)  # Top 10 résultats
distances, indices

**With embedding**

In [117]:
type(embeddings)

numpy.ndarray

In [ ]:
# Création de l'index FAISS
dim = embeddings.shape[1]
print("dim", dim)
index = faiss.IndexFlatIP(dim)  # Produit Scalaire
#faiss.IndexFlatL2 ...

print(type(index))
index.add(embeddings.astype(np.float32))

# Recherche
search_text = "chien"
search_vector = model.encode(search_text)
distances, indices = index.search(query_embedding, k=6)  # Top 10 résultats
distances, indices

dim 1024
<class 'faiss.swigfaiss_avx2.IndexFlatIP'>


(array([[ 6.021308 ,  4.0883536, -0.887233 , -2.646186 , -3.7058098,
         -6.065552 ]], dtype=float32),
 array([[3, 1, 4, 5, 0, 2]]))

## Example from Medium
https://medium.com/loopio-tech/how-to-use-faiss-to-build-your-first-similarity-search-bf0f708aa772

In [ ]:
data = [['Where are your headquarters located?', 'location'],
['Network Access Control?', 'networking'],
['Throw my cellphone in the water', 'random'],
['Address', 'location']]
df_ = pd.DataFrame(data, columns = ['text', 'category'])

text = df_['text']
encoder = model
vectors = encoder.encode(text)

vector_dimension = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

search_text = 'where is your office?'
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

k = index.ntotal
print('k', k)
distances, ann = index.search(_vector, k=k)

results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
results

k 4


,distances,ann
0,0.294170,0
1,0.863272,3
2,1.523529,1
3,1.835073,2


## Exemple GPT

TF-IDF en matrice creuse suivi d'une réduction de dimension puis de recherche de similarité avec FAISS

In [155]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import faiss

# 1. Quelques documents exemple
documents = [
    "le chat dort sur le canapé",
    "le chien joue dans le jardin",
    "le chat et le chien sont amis",
    "la pluie tombe sur le jardin",
    "le canapé est confortable",
    "Un avion est en train de décoller.",
    "Un homme joue d'une grande flûte.",
    "Un homme étale du fromage râpé sur une pizza.",
    "Une personne jette un chat au plafond.",
    "Une personne est en train de plier un morceau de papier.",
    "Un italien prépare le repas avec du fromage"
]

# 2. Vectorisation TF-IDF
vectorizer = TfidfVectorizer()
sparse_matrix = vectorizer.fit_transform(documents)  # Résultat : matrice creuse (csr_matrix)

# 3. Réduction de dimension pour éviter une trop grande densité (ex : 50 dim)
svd = TruncatedSVD(n_components=17)
dense_matrix = svd.fit_transform(sparse_matrix).astype('float32')  # FAISS attend du float32

# 4. Création de l’index FAISS
dimension = dense_matrix.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(dense_matrix)  # Ajoute les vecteurs

# 5. Recherche de similarité pour une requête
# requete = ["un chien dort dans le jardin"]
requete = ["une femme mange le camembert"]
query_tfidf = vectorizer.transform(requete)
query_dense = svd.transform(query_tfidf).astype('float32')

# 6. Recherche des 3 documents les plus similaires
k = len(documents)
distances, indices = index.search(query_dense, k)

# 7. Affichage des résultats
print("Requête :", requete[0])
print("\nDocuments les plus similaires :")
for i, idx in enumerate(indices[0]):
    print(f"{i+1}. {documents[idx]} (distance : {distances[0][i]:.4f})")


Requête : une femme mange le camembert

Documents les plus similaires :
1. le chat dort sur le canapé (distance : 0.6126)
2. le chien joue dans le jardin (distance : 0.6391)
3. le chat et le chien sont amis (distance : 0.7095)
4. Un homme joue d'une grande flûte. (distance : 0.7996)
5. Une personne jette un chat au plafond. (distance : 0.8432)
6. le canapé est confortable (distance : 0.8716)
7. Un homme étale du fromage râpé sur une pizza. (distance : 0.8984)
8. Une personne est en train de plier un morceau de papier. (distance : 0.9672)
9. la pluie tombe sur le jardin (distance : 0.9793)
10. Un italien prépare le repas avec du fromage (distance : 1.0215)
11. Un avion est en train de décoller. (distance : 1.3054)
